### Just Giving API POC (Kindness Walks)

In [0]:
import requests
import json

from datetime import datetime
from datetime import date
import requests
import sys
from math import floor
from time import time

import pandas as pd
from operator import itemgetter

from requests.auth import HTTPBasicAuth

import datetime as dt


In [0]:
#Widget to grab Slice
dbutils.widgets.text("WindowStart","")
dbutils.widgets.text("WindowEnd","")
dbutils.widgets.text("EventID","")

Set Parameters

In [0]:
#Secrets Should be in KeyVault
#SalesForce
authentication_base_URI = 
rest_base_URI = 
sub_domain = 
clientid = 
clientsecret =
external_key = 
import_url= 

#JustGiving
username = 
password = 
charity_id = 
app_id = 

In [0]:
#Set parameters for this notebook
datefrom = datetime.strptime(dbutils.widgets.get("WindowStart"), '%Y-%m-%d')
dateto = datetime.strptime(dbutils.widgets.get("WindowEnd"), '%Y-%m-%d')
datefromstr = dbutils.widgets.get("WindowStart")
datetostr = dbutils.widgets.get("WindowEnd")
event_id = dbutils.widgets.get("EventID")

Sales Force Functions
1. Generate Access Tokens. Returns access tokens and how long valid for. Valid for not required; new token generated every time
1. Import data

In [0]:
#returns access token and how long it is valid for, the valid for is not required as we will for a new token everytime
def generate_access_token(clientid: str, clientsecret: str) -> str:
    subdomain = sub_domain
    auth_base_url = authentication_base_URI
    headers = {'content-type': 'application/json'}
    payload = {
      'grant_type': 'client_credentials',
      'client_id': clientid,
      'client_secret': clientsecret
    }
    authentication_response = requests.post(
        url=auth_base_url, data=json.dumps(payload), headers=headers
    ).json()

    if 'access_token' not in authentication_response:
        raise Exception(
          f'Unable to validate (ClientID/ClientSecret): {repr(authentication_response)}'
        )
    access_token = authentication_response['access_token']
    expires_in = time() + authentication_response['expires_in']

    return access_token, expires_in

In [0]:
generate_access_token(clientid, clientsecret)

In [0]:
def datetime_converter(value: datetime) -> str:
    if isinstance(value, datetime):
        return value.__str__() 

In [0]:
def get_batch_size(record: dict) -> int:
    batch = json.dumps({'items': record}, default=datetime_converter)
    return floor(4000 / (sys.getsizeof(batch) / 1024))

In [0]:
def import_data(clientid: str, clientsecret: str,  data: list) -> None:
    access_token, expires_in = generate_access_token(clientid, clientsecret)
    subdomain = sub_domain
    rest_url = rest_base_URI
    headers = {'authorization': f'Bearer {access_token}'}
    
    batch_size = get_batch_size(data[0])
    print(batch_size)
    for batch in range(0, len(data), batch_size):
        if expires_in < time() + 60:
             access_token,expires_in = generate_access_token(clientid, clientsecret)
        batch_data = data[batch:batch + batch_size]
        insert_request = requests.post(
            url=import_url,
            data=json.dumps({'items': batch_data}, default=datetime_converter),
            headers=headers
        )

        if insert_request.status_code not in (200, 202):
            raise Exception(f'Insertion failed with message: {insert_request.json()}')
    insert_request.close() 

Extract from JustGiving and Import into Salesforce. Set to extract daily. Loops through date range given by day

In [0]:
 headers = {'Content-type': 'application/json'}


# The size of each step in days
day_delta_from = dt.timedelta(days=1)
day_delta_to = dt.timedelta(days=1)



#start_date = dt.datetime.strptime(datefrom, '%Y-%m-%d').date()
daystoprocess = abs((datefrom - dateto).days)
end_date = datefrom + daystoprocess*day_delta_from

for i in range((dateto - datefrom).days):
    #print(start_date + i*day_delta_from)
    datefromstr= (datefrom + i*day_delta_from).strftime('%Y-%m-%d')
    datetostr = (datefrom + i*day_delta_to).strftime('%Y-%m-%d')
    print(datefrom)
    url= 'https://dataapi.justgiving.com/'+app_id+'/v1/charity/'+charity_id+'/events/'+event_id+'/pages/created/'+datefromstr+';'+datetostr+''
    
   
    print(url)
    
    try:
        response1 = requests.get(url,headers =headers, auth = HTTPBasicAuth(username, password)) ####Do Not Delete
        
        #Nomalise nested Jsons
        json_data = json.loads(response1.text)
        json_data3 = pd.json_normalize(json_data)
        
        #rename columns
        json_data3.rename(columns = {'DonationsAndCharges.NumberOnlineDonations':'DonationsAndCharges_NumberOnlineDonations'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.NumberSmsDonations':'DonationsAndCharges_NumberSmsDonations'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.OfflineAmount':'DonationsAndCharges_OfflineAmount'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.TargetAmount':'DonationsAndCharges_TargetAmount'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.TotalDonations':'DonationsAndCharges_TotalDonations'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.TotalDonationsAndGiftAidMinusEstimatedFees':'DonationsAndCharges_TotalDonationsAndGiftAidMinusEstimatedFees'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.TotalDonationsPlusGiftAid':'DonationsAndCharges_TotalDonationsPlusGiftAid'}, inplace = True)
        json_data3.rename(columns = {'DonationsAndCharges.TotalSmsDonations':'DonationsAndCharges_TotalSmsDonations'}, inplace = True)
        json_data3.rename(columns = {'Event.Category':'Event_Category'}, inplace = True)
        json_data3.rename(columns = {'Event.CustomCode1':'Event_CustomCode1'}, inplace = True)
        json_data3.rename(columns = {'Event.CustomCode2':'Event_CustomCode2'}, inplace = True)
        json_data3.rename(columns = {'Event.CustomCode3':'Event_CustomCode3'}, inplace = True)
        json_data3.rename(columns = {'Event.Date':'Event_Date'}, inplace = True)
        json_data3.rename(columns = {'Event.ExpiryDate':'Event_ExpiryDate'}, inplace = True)
        json_data3.rename(columns = {'Event.Id':'Event_Id'}, inplace = True)
        json_data3.rename(columns = {'Event.IsOverseas':'Event_IsOverseas'}, inplace = True)
        json_data3.rename(columns = {'Event.IsPromoted':'Event_IsPromoted'}, inplace = True)
        json_data3.rename(columns = {'Event.IsUserCreated':'Event_IsUserCreated'}, inplace = True)
        json_data3.rename(columns = {'Event.Name':'Event_Name'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Country':'Fundraiser_Address_Country'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Line1':'Fundraiser_Address_Line1'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Line2':'Fundraiser_Address_Line2'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Postcode':'Fundraiser_Address_Postcode'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Region':'Fundraiser_Address_Region'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Address.Town':'Fundraiser_Address_Town'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.ConnectedBenefit':'Fundraiser_ConnectedBenefit'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Email':'Fundraiser_Email'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.FirstName':'Fundraiser_FirstName'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.FurtherContact':'Fundraiser_FurtherContact'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.LastName':'Fundraiser_LastName'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.Title':'Fundraiser_Title'}, inplace = True)
        json_data3.rename(columns = {'Fundraiser.UserId':'Fundraiser_UserId'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.DateOfBirth':'InMemoriam_DateOfBirth'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.DateOfDeath':'InMemoriam_DateOfDeath'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.FirstName':'InMemoriam_FirstName'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.Gender':'InMemoriam_Gender'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.Id':'InMemoriam_Id'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.IsInMemoriam':'InMemoriam_IsInMemoriam'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.LastName':'InMemoriam_LastName'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.Name':'InMemoriam_Name'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.Relationship':'InMemoriam_Relationship'}, inplace = True)
        json_data3.rename(columns = {'InMemoriam.Town':'InMemoriam_Town'}, inplace = True)
        json_data3.rename(columns = {'OrganisationPortal.Name':'OrganisationPortal_Name'}, inplace = True)
        json_data3.rename(columns = {'OrganisationPortal.Url':'OrganisationPortal_Url'}, inplace = True)
        json_data3.rename(columns = {'ReferralWebsite.Name':'ReferralWebsite_Name'}, inplace = True)
        json_data3.rename(columns = {'ReferralWebsite.Url':'ReferralWebsite_Url'}, inplace = True)
        json_data3.rename(columns = {'Team.Members':'Team_Members'}, inplace = True)
        json_data3.rename(columns = {'Team.Name':'Team_Name'}, inplace = True)
        json_data3.rename(columns = {'Team.Url':'Team_Url'}, inplace = True)      
        
        #extract only required fields
        json_data4= json_data3[['Appeal','Fundraiser_UserId','Fundraiser_FirstName','Fundraiser_LastName','Fundraiser_Email','Status','Url','Event_Name','DonationsAndCharges_TotalDonations','Team_Url','Team_Name']]
       
        # convert json into dict and format into format acceptable for sales force
        json_data5 = json_data4.to_dict(orient='records')
     
        #import into salesforce
        import_data(clientid, clientsecret,json_data5)


    except IndexError:
        print('No Data for Date')

In [0]:
json_data3

In [0]:
json_data4